Data.gov
Source: https://catalog.data.gov/dataset/consumer-complaint-database#topic=consumer_navigation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
# Used to see all rows of a data frame.
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
complaints = pd.read_csv('Consumer_Complaints.csv')

In [ ]:
print( len(complaints))
complaints.head()



In [ ]:
cols_dropped = ['Consumer complaint narrative', 'Company public response', 'Tags', 
                'Consumer consent provided?,', 'Submitted via', 'Timely response?', 
                'Complaint ID'],

Investigate:
* Most complained about products + sub-products
* Most complained about issues + sub-issues
* What does it mean to get a "timely" response? Website says 97% timeley responses, but what does that mean?
** Source: https://www.consumerfinance.gov/data-research/consumer-complaints/
** Not much clearer on the company's data field github page either: https://cfpb.github.io/api/ccdb/fields.html
* How many complaints were sent to company the same day they were submitted by the user? Was this only b/c of weekends? Or 
    longer delay for certain things? 
* Use the timeclass object to chedk for Fridays & other analysis. Will need to convert date columns to timeclass or pandas
    timeclass

Next steps:
* Basecamp to display zip codes with most complaints? What types of companies are most complained about by state?

In [ ]:
cols_dropped = ['Consumer complaint narrative', 'Tags', 'Complaint ID']
complaints.drop(cols_dropped, axis = 1, inplace = True)

In [ ]:
print( len(complaints['Company public response'].unique()) )
print(len(complaints['Company response to consumer'].unique()) )

In [ ]:
print ("Shape: ", complaints.shape)
complaints.head()

In [ ]:
# Column renaming from Pandas cheat sheet: https://www.dataquest.io/blog/pandas-cheat-sheet/
complaints.columns = ['date_received', 'product', 'sub_product', 
                      'issue', 'sub_issue', 
                      'com_public_response', 
                      'company', 'state', 'zip', 
                      'consent_provided?', 'submitted_via',
                      'date_to_company', 'comp_response_to_consumer',
                      'timely_response?', 'consumer_disputed?']

complaints.head()

In [ ]:
complaints.drop(complaints[''])

In [ ]:
complaints['date_received'] = pd.to_datetime(complaints['date_received'], format = '%m/%d/%Y') a
complaints['date_receivedd'] = pd.to_datetime(complaints['date_to_company'], format = '%m/%d/%Y')

In [ ]:
print( complaints.info() )
print( '\n')
print( complaints.dtypes)

In [ ]:
# print("Products: ", len(complaints['product'].unique()))
# print("Sub-products: ", len(complaints['sub-product'].unique() ))





In [ ]:
# Remove "I do not know" label from all subproducts"
complaints.drop(complaints[complaints['sub_product'] == 'I do not know'].index, inplace = True)

In [ ]:
# Display products by # of complaints
products = complaints['product'].value_counts()

print(products)
# print(type(products))
complaints['product'].value_counts().plot(kind='bar')

There are several product categories that are redundant:
* Credit card / credit card or prepaid card / prepaid card
* Money transfers / money transfer, virtual currency, or money service / virtual currency
* Payday loan / payday loan, title loan, orpersonal loan / consumer loan

The following section sorts the aggregate product categories into other product categories that already exist.

In [ ]:
complaints['product'].value_counts()



In [ ]:
complaints.loc[complaints['product'] == 'Consumer Loan'] = 'Consumer loan' # for consistency's sake

In [ ]:
# This cell was used as aid to help sort sub_products into correct product groups, out of mixed groups

## Check Product 1
credit_or_prepaid_bool = complaints['product'] == 'Credit card or prepaid card'
credit_or_prepaid_cards = complaints[credit_or_prepaid_bool]
print("Credit or prepaid card sub_products:", 
      '\n', 
      credit_or_prepaid_cards['sub_product'].value_counts(),
     '\n')

## Check Product 1
## No sub_product for credit cards
credit_card_bool = complaints['product'] == 'Credit card'
credit_cards = complaints[credit_card_bool]

print("Credit cards sub_products:", 
      '\n', 
      credit_cards['sub_product'].value_counts(),
     '\n')

# Check Product 1
prepaid_bool = complaints['product'] == 'Prepaid card'
prepaid_cards = complaints[prepaid_bool]
print("Prepaid cards sub_products:",
      '\n',
      prepaid_cards['sub_product'].value_counts(),
      '\n')

mt_vc_ms_bool = complaints['product'] == 'Money transfer, virtual currency, or money service'
mt_vc_ms_subproducts = complaints[mt_vc_ms_bool]
print("Money transfer, virtual currency, or money service sub_products:", 
      '\n', 
      mt_vc_ms_subproducts['sub_product'].value_counts(),
     '\n')

In [ ]:
# This cell eliminates redundant product groups
## 'Credit card and prepaid card'
# Setting with copy warning solution found here: https://www.dataquest.io/blog/settingwithcopywarning/


# Credit & prepaid cards
complaints.loc[complaints['sub_product'] == 'General-purpose credit card or charge card', 'product'] = 'Credit card'
complaints.loc[complaints['sub_product'] == 'Store credit card', 'product'] = 'Credit card'

complaints.loc[complaints['sub_product'] == 'General-purpose prepaid card', 'product'] = 'Prepaid card'
complaints.loc[complaints['sub_product'] == 'Government benefit card', 'product'] = 'Prepaid card'
complaints.loc[complaints['sub_product'] == 'Payroll card', 'product'] = 'Prepaid card'
complaints.loc[complaints['sub_product'] == 'Gift card', 'product'] = 'Prepaid card'
complaints.loc[complaints['sub_product'] == 'Student prepaid card', 'product'] = 'Prepaid card'

# Money transfers, virtual currency, or money service
# Display sub_products in 'Virtual currency' product group. Only 2 types, not many data points.
print ( complaints.loc[complaints['product'] == 'Virtual currency', 'sub_product'].value_counts() )

complaints.loc[complaints['sub_product'] == 'Virtual currency', 'product'] = 'Virtual currency'
complaints[complaints['sub_product'] == 'Virtual currency'] # remove when complete

complaints.loc[complaints['sub_product'] == 'Domestic (US) money transfer', 'product'] = 'Money transfers'
complaints.loc[complaints['sub_product'] == 'International money transfer', 'product'] = 'Money transfers'
complaints.loc[complaints['sub_product'] == 'Foreign currency exchange', 'product'] = 'Money transfers'
complaints.loc[complaints['sub_product'] == 'Mobile or digital wallet', 'product'] = 'Money transfers'

# Rename trimmed product group 
complaints.loc[complaints['product'] == 'Money transfer, virtual currency, or money service', 'product'] = 'Money service'

# Payday & consumer loans
complaints.loc[(complaints['sub_product'] == 'Payday loan') &  
               (complaints['product'] == 'Payday loan, title loan, or personal loan'), 'product'] = 'Payday loan'

complaints.loc[(complaints['sub_product'] == 'Personal line of credit') &  
               (complaints['product'] == 'Payday loan, title loan, or personal loan'), 'product'] = 'Consumer loan'

complaints.loc[(complaints['sub_product'] == 'Installment loan') &  
               (complaints['product'] == 'Payday loan, title loan, or personal loan'), 'product'] = 'Consumer loan'

complaints.loc[(complaints['sub_product'] == 'Title loan') &  
               (complaints['product'] == 'Payday loan, title loan, or personal loan'), 'product'] = 'Consumer loan'

complaints.loc[(complaints['sub_product'] == 'Pawn loan') &  
               (complaints['product'] == 'Payday loan, title loan, or personal loan'), 'product'] = 'Consumer loan'

# Credit reporting
complaints.loc[complaints['product'] == 'Credit reporting, credit repair services, or other personal consumer reports', 
               'product'] = 'Credit reporting'

In [ ]:
# print ( complaints.loc[complaints['product'] == 'Money transfers', 'sub_product'].value_counts() )



complaints.loc[complaints['product'] == 'Credit reporting', 'sub_product'].value_counts()

# print ( complaints.loc[complaints['product'] == 'Payday loan, title loan, or personal loan', 'sub_product'].value_counts() ) 
# print('\n')
# print ( complaints.loc[complaints['product'] == 'Consumer loan', 'sub_product'].value_counts() ) 
# print('\n')
# print ( complaints.loc[complaints['product'] == 'Payday loan', 'sub_product'].value_counts() ) 


In [ ]:
complaints['product'].value_counts()

To do:
* Then, create exploratory graphic that lays them next to eachother


In [ ]:
# REVISION: Do this by product category manually, then find a way to automate it later
## Create a dict w/ the product:sub_product key:value pairs?

mortgage_bool = complaints['product'] == 'Mortgage'
mortgage_subproducts = complaints[mortgage_bool]['sub_product'].value_counts()

debt_col_bool = complaints['product'] == 'Debt collection'
debt_col_subproducts = complaints[debt_col_bool]['sub_product'].value_counts()

credit_reporting_bool = complaints['product'] == 'Credit reporting'
credit_rep_subproducts = complaints[credit_reporting_bool]['sub_product'].value_counts()

credit_card_bool = complaints['product'] == 'Credit card'
credit_card_subproducts = complaints[credit_card_bool]['sub_product'].value_counts

bank_account_bool = complaints['product'] == 'Bank account or service'
bank_account_subproducts = complaints[bank_account_bool]['sub_product'].value_counts

student_loan_bool = complaints['product'] == 'Studen loan'
studen_loan_subproducts = complaints[student_loan_bool]['sub_product'].value_counts()

checking_savings_bool = complaints['product'] == 'Checking or savings account'
checking_savings_subproducts = complaints[checking_savings_bool]['sub_product'].value_counts()

consumer_loan_bool = complaints['product'] == 'Consumer Loan'
consumer_loan_subproducts = complaints[consumer_loan_bool]['sub_product'].value_counts()

vehicle_loan_bool = complaints['product'] == 'Vehicle loan or lease'
vehicle_loan_subproducts = complaints[vehicle_loan_bool]['sub_product'].value_counts()

## Check Product 2
mt_bool = complaints['product'] == 'Money transfers'
mt_subproducts = complaints[mt_bool]['sub_product'].value_counts()

ms_bool = complaints['product'] == 'Money service'
ms_subproducts = complaints[ms_bool]['sub_product'].value_counts()

payday_bool = complaints['product'] == 'Payday loan'
payday_subproducts = complaints[payday_bool]['sub_product'].value_counts()

mt_bool = complaints['product'] == 'Money transfers'
mt_subproducts = complaints[mt_bool]['sub_product'].value_counts()

prepaid_bool = complaints['product'] == 'Prepaid card'
prepaid_subproducts = complaints[prepaid_bool]['sub_product'].value_counts()

other_bool = complaints['product'] == 'Other financial service'
other_subproducts = complaints[other_bool]['sub_product'].value_counts()

vc_bool = complaints['product'] == 'Virtual currency'
vc_subproducts = complaints[vc_bool]['sub_product'].value_counts()








In [ ]:
## Check Product 2
ms_bool = complaints['product'] == 'Money service'
ms_subproducts = complaints[ms_bool]['sub_product'].value_counts()
ms_subproducts

In [ ]:
products = complaints['product'].value_counts()
products = products.to_frame() # Otherwise products is a series and additional column cannot be added

# Calculate number of complaints by % of total
products.columns = ['num_complaints']
products['perc_total'] = products['num_complaints']/sum(products['num_complaints'])

print(products)

In [ ]:
sub_products = complaints['sub_product'].value_counts()
sub_products = sub_products.to_frame() # Otherwise sub_products is a series and additional column cannot be added

# Calculate number of complaints by % of total
sub_products.columns = ['num_complaints']
sub_products['perc_total'] = sub_products['num_complaints']/sum(sub_products['num_complaints'])

print(sub_products)

In [ ]:
print('Credit report', '\n', complaints.loc[complaints['product'] == 'Credit reporting', 'sub_product'].value_counts() )
print('\n')

print('Mortgage', '\n', complaints.loc[complaints['product'] == 'Mortgage', 'sub_product'].value_counts() )
print('\n')

print('Debt collection', '\n', complaints.loc[complaints['product'] == 'Debt collection', 'sub_product'].value_counts() )
print('\n')

print('Credit card', '\n', complaints.loc[complaints['product'] == 'Credit card', 'sub_product'].value_counts() )
print('\n')

print('Bank account or service', '\n', complaints.loc[complaints['product'] == 'Bank account or service', 'sub_product'].value_counts() )
print('\n')

# I wonder why complaints for fixed mortgages are twice as high as ARMs?
# Difficult to know with the "other mortgage" category.

In [ ]:
prod_greater_5 = products[products['perc_total'] > 0.05]
prod_less_5 = products[products['perc_total'] < 0.05]

# Prod greater than 5%
# prod_greater_5_name = prod_greater_5

prod_greater_5_type = prod_greater_5.index.values

# plt.bar(prod_greater_5_type, prod_greater_5['perc_total'])
# # plt.tick_params(bottom = "off", top = "off", left = "off", right = "off")


fig, ax = plt.subplots()
ax.bar(prod_greater_5_type, prod_greater_5['perc_total'], color=[246/255, 168/255, 78/255])

ax.spines["left"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)



# Function definition from: https://stackoverflow.com/questions/28931224/adding-value-labels-on-a-matplotlib-bar-chart
def add_value_labels(ax, spacing=5):
    """Add labels to the end of each bar in a bar chart.

    Arguments:
        ax (matplotlib.axes.Axes): The matplotlib object containing the axes
            of the plot to annotate.
        spacing (int): The distance between the labels and the bars.
    """

    # For each bar: Place a label
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        label = "{:.0f}%".format(y_value*100)

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(0, space),          # Vertically shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally center label
            va=va)                      # Vertically align label differently for
                                        # positive and negative values.


# Call the function above. All the magic happens there.
add_value_labels(ax)

# Turn off tick labels
# Source: https://stackoverflow.com/questions/37039685/hide-axis-values-in-matplotlib
ax.set_yticklabels([])


plt.yticks([])

plt.title('Top 5 most complained about products', fontweight = 'bold', ha = 'center', pad = 25)
plt.xticks(rotation = 90)


fig.tight_layout()
plt.show()

prod_greater_5['perc_total'].plot(kind = 'bar')

In [ ]:
print( complaints.loc[complaints['product'] == 'Credit reporting', 'issue'].value_counts() )
print('\n')
print( complaints.loc[complaints['product'] == 'Mortgage', 'issue'].value_counts() )
print('\n')
print( complaints.loc[complaints['product'] == 'Debt collection', 'issue'].value_counts() )
print('\n')
print( complaints.loc[complaints['product'] == 'Credit card', 'issue'].value_counts() )
print('\n')
print( complaints.loc[complaints['product'] == 'Bank account or service', 'issue'].value_counts() )


In [ ]:
# Display sub-products by % of total complaints by sub-products
# REVISION: Add threshold variable

sub_prod_greater_3 = sub_products[sub_products['perc_total'] > 0.03]
sub_prod_less_3 = sub_products[sub_products['perc_total'] < 0.03]

sub_prod_greater_3['perc_total'].plot(kind = 'bar')